In [1]:
import pandas as pd
import numpy as np

In [16]:
df_ROS = pd.read_csv('../../../datasets/RecurrenceOfSurgery.csv')
df_ROS[:3]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0
2,2,3PT,53.0,3,NaN,0.0,0,0,1,0,...,0,67.5,27.3,10.2,0,-1.77,1749.0,4,Down,0


In [3]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

#### 목표변수 
- 목표변수 : 연령
- 설명변수 : 입원기간, 신장, Large Lymphocyte

In [17]:
# 목표변수와 설명변수 축출
df_ROS_extract = df_ROS[['연령', '입원기간', '신장', 'Large Lymphocyte']]
df_ROS_extract[:3]

,연령,입원기간,신장,Large Lymphocyte
0,66,2,163,22.8
1,47,1,171,44.9
2,39,1,178,53.0


In [5]:
df_ROS_extract.isnull().sum()

연령                  0
입원기간                0
신장                  0
Large Lymphocyte    0
dtype: int64

In [6]:
# 목표변수(target)와 설명변수(label) 분할하기
target = df_ROS_extract['연령']
labels = df_ROS_extract[['입원기간', '신장', 'Large Lymphocyte']]
target.shape, labels.shape

((1894,), (1894, 3))

In [7]:
# 모델 학습
from sklearn.linear_model import LinearRegression

In [8]:
model = LinearRegression()
model.fit(labels, target)

LinearRegression()

In [9]:
# 평가
target_predict = model.predict(labels) 
len(target_predict)

1894

In [10]:
from sklearn.metrics import r2_score
r2_score(target, target_predict)

0.042823545290537446

In [12]:
df_ROS_extract[50:55]

,연령,입원기간,신장,Large Lymphocyte
50,55,1,169,31.5
51,38,2,161,26.7
52,29,2,168,36.9
53,40,1,185,38.6
54,38,2,168,86.1


In [13]:
# 51번째의 2, 161, 26.7 : 외부에서 입력한 설명변수 값들
temp_label = [[2, 161, 26.7]]

In [14]:
model.predict(temp_label)

C:\Users\01-02\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([43.432433])

In [15]:
# 서비스 배포
import pickle

In [18]:
with open('../../../datasets/RecurrenceOfSurgery_Regression.pkl', 'wb') as regression_file :
    pickle.dump(obj=model, file=regression_file)
    pass

In [19]:
# 모델 일반화 확인
model.intercept_, model.coef_
# y = (0.23801685 * 입원기간) + (-0.29248605 * 신장) + (0.01188728 * Large Lymphocyte) + 89.72926381099202

(89.72926381099202, array([ 0.23801685, -0.29248605,  0.01188728]))

In [20]:
# [[2, 161, 26.7]]
(0.23801685 * 2) + (-0.29248605 * 161) + (0.01188728 * 26.7) + 89.72926381099202
# array([43.432433])

43.432433836992026